In [2]:
suppressPackageStartupMessages(require(tidyverse))
suppressPackageStartupMessages(require(data.table))


Warning message:
“package ‘ggplot2’ was built under R version 3.6.2”Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.2”Warning message:
“package ‘dplyr’ was built under R version 3.6.2”Warning message:
“package ‘forcats’ was built under R version 3.6.2”

In [1]:
ukb_dir   <- '/oak/stanford/groups/mrivas/ukbb/24983'
imp_dir   <- file.path(ukb_dir, 'imp/pgen/var_QC_noHWE')
cal_ID_f  <- file.path(ukb_dir, 'cal/pgen/ukb24983_cal_cALL_v2_hg19.IDs.tsv')
hla_ID_f  <- file.path(ukb_dir, 'hla/pgen/ukb_hla_v3.IDs.tsv')
cnv_ID_f  <- file.path(ukb_dir, 'cnv/pgen/cnv.IDs.tsv')

merged_dir <- file.path(ukb_dir, 'array_imp_combined/pgen_v2')


#### Read the ID mapping file

In [4]:
imp_ID_df <- bind_rows(lapply(c(1:22, 'X', 'XY'), function(c){fread(
file.path(imp_dir, paste0('ukb24983_imp_chr', c, '_v3_QC_noHWE.IDs.tsv')),
colClasses=c(CHROM="character")
)}))


In [5]:
non_imp_ID_df <- bind_rows(lapply(c(cal_ID_f, hla_ID_f, cnv_ID_f), 
function(f){fread(f,colClasses=c(CHROM="character")
)}))


In [6]:
ID_map <- bind_rows(imp_ID_df, non_imp_ID_df)


#### Read the bim file from merge

In [7]:
bim_short <- fread(file.path(merged_dir, 'ukb24983_ukb24983_cal_hla_cnv_imp.shortnames.bim'))
names(bim_short) <- c('CHR', 'ID', 'CM', 'POS', 'ALT', 'REF')


#### join

In [8]:
df <- bim_short %>% mutate(sort_order = 1:n()) %>%
inner_join(ID_map %>% select(ID, full_ID), by='ID')


In [9]:
bim_short %>% dim() %>% print()
df %>% dim() %>% print()


[1] 7012330       6
[1] 7012330       8


In [10]:
df %>% arrange(sort_order) %>% 
select(CHR, full_ID, CM, POS, ALT, REF) %>%
fwrite(
    file.path(merged_dir, 'ukb24983_ukb24983_cal_hla_cnv_imp.bim'),
    col.names=F, sep='\t'
)


In [11]:
df %>% select(full_ID) %>% 
mutate(ID_len = str_length(full_ID)) %>% select(ID_len) %>% max() %>% print()


[1] 181
